In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import GoogleGenerativeAI
from langchain_openai import ChatOpenAI
from inspect import signature
from yaml import safe_load
import pandas as pd 
import tqdm
import re
import json
from pathlib import Path
import os
import faiss
import gc
from typing import Tuple
import dotenv
import time
OUTPUT_DIR = Path("test_results")
today = pd.Timestamp.now().strftime("%Y-%m-%d")

dotenv.load_dotenv()


False

In [9]:
class XRIFGenerator:
    def __init__(self, 
                 prompt_template: str,
                 waypoints_csv: str,
                 language_model_type = "Ollama",
                 embed_model_type = "Ollama",
                 language_model: str = "deepseek-r1:1.5b",
                 embed_model: str = "deepseek-r1:1.5b",
                 chat_kwargs: dict = {"num_predict": 500, "temperature": 0.2, "top_p": 0.2},
                 fetch_all: bool = True):
        self.waypoints_list = []
        self.kwargs = chat_kwargs
        self.prompt_template = prompt_template
        self.waypoints_csv = waypoints_csv
        self.language_model_type = language_model_type
        self.x_coord = 0
        self.y_coord = 0
        print(f"Loading model: {language_model}...")
        
        if embed_model_type == "Ollama":
            embed = OllamaEmbeddings(model = embed_model)
        elif embed_model_type == "HuggingFace":
            embed = HuggingFaceEmbeddings(model_name = embed_model)
        else:
            raise Exception("Invalid embed_model_type")

        if language_model_type == "Ollama":
            self.llm = OllamaLLM(model=language_model, **self.filter_kwargs_for_OllamaLLM(self.kwargs))
        elif language_model_type == "google":
            if os.getenv("GOOGLE_API_KEY"):
                self.llm = GoogleGenerativeAI(model=language_model, **self.kwargs)
            else:
                raise Exception("GOOGLE_API_KEY not found in environment variables")
        elif language_model_type == "OpenAI":
            try: 
                self.llm = ChatOpenAI(model=language_model, **self.filter_kwargs_for_OpenAI(self.kwargs))
            except Exception as e:
                print(f"Error loading OpenAI model: {e}")
        else:
            raise Exception("Invalid language_model_type")

        self.output_parser = StrOutputParser()
        waypoints = pd.read_csv(self.waypoints_csv)
        waypoint_cols = ['Location', 'X co-ordinate', 'Y co-ordinate', 'Floor', 'Section', 'Keywords']
        given_waypoints_cols = list(waypoints.columns)
        if given_waypoints_cols != waypoint_cols:
            raise Exception("Columns do not match, please provide a new dataset")
        
        num_rows = len(waypoints)
        print(f"Processing {num_rows} waypoints...")
        with tqdm.tqdm(total=num_rows) as pbar:
            for index, row in waypoints.iterrows():
                self.waypoints_list.append(Document(id = f"Waypoint_{index}", page_content=f"A waypoint called {row['Location']} exists at X-Coordinate, {row['X co-ordinate']}, and Y-Coordinate, {row['Y co-ordinate']}, apart of Section, {row['Section']}. Words associated with the waypoint, {row['Location']} are {row['Keywords']}."))
                pbar.update(1)
        
        index = faiss.IndexFlatL2(len(embed.embed_query("Hello World!")))

        self.vector_store = FAISS(
            embedding_function=embed,
            index=index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={}            
        )
        
        print("Adding waypoints to vector store...")
        self.vector_store.add_documents(self.waypoints_list)
        print("Creating retriever...")
        k = len(self.waypoints_list) if fetch_all else 5
        
        self.retriever = self.vector_store.as_retriever(
            search_type = "mmr",
            search_kwargs = {'k': k, 'fetch_k': len(self.waypoints_list)},
        )

        with open(self.prompt_template, 'r') as file:
            prompt_dict = safe_load(file)
            self.prompt = prompt_dict['prompt']
            self.prompt_name = prompt_dict['prompt_name']
        
        print("Creating prompt template...")
        self.prompt_template = PromptTemplate(template=self.prompt, input_variables=["documents", "query", "starting_location"])

        print("Creating RAG chain...")
        self.rag_chain = self.prompt_template | self.llm | self.output_parser
        
    def filter_kwargs_for_OllamaLLM(self, kwargs: dict) -> dict:
        chatollama_params = signature(OllamaLLM.__init__).parameters
        return {k: v for k, v in kwargs.items() if k in chatollama_params}
    
    def filter_kwargs_for_OpenAI(self, kwargs: dict) -> dict:
        chatopenai_params = signature(ChatOpenAI.__init__).parameters
        return {k: v for k, v in kwargs.items() if k in chatopenai_params}
    
    def generate_xrif_with_deepseek_model(self, query: str):
        xrif_response = None
        documents = self.retriever.invoke(query)
        num_waypoints_injected = len(documents)
        doc_texts = "\n".join([doc.page_content for doc in documents])

        rendered_prompt = self.prompt_template.format(documents = doc_texts, query= query, starting_location = f"X-Coordinate: {self.x_coord}, Y-Coordinate: {self.y_coord}")

        llm_response = self.rag_chain.invoke({"documents": doc_texts, "query": query, "starting_location": f"X-Coordinate: {self.x_coord}, Y-Coordinate: {self.y_coord}"})

        xrif_response = re.sub(r"<think>.*?</think>", "", llm_response, flags=re.DOTALL)

        xrif_response = re.sub(r"\n", "", xrif_response)

        xrif_response = re.sub(r"\\n", "", xrif_response)

        start = xrif_response.find('{')
        end = xrif_response.rfind('}')

        if start != -1 and end != -1:
            xrif_response = xrif_response[start:end+1]
            try:
                xrif_response = json.loads(xrif_response)
            except json.JSONDecodeError as e:
                print("Error parsing JSON response")

        
        return xrif_response, llm_response, rendered_prompt, doc_texts, num_waypoints_injected
    
    def batch_test_run(self, df: pd.DataFrame, output_folder: Path):
        num_rows = len(df)
        print(f"Processing {num_rows} test prompts...")
        with tqdm.tqdm(total=num_rows) as pbar:
            for index, row in df.iterrows():
                now = pd.Timestamp.now().strftime("%Y-%m-%d-%H-%M-%S")
                dump_json = {}
                error_message = 'No Error'
                note = ''
                query = row['Prompt']
                self.x_coord = row['Starting X']
                self.y_coord = row['Starting Y']
                expected_response = row['Expected Response']
                llm_response, xrif_response = self.generate_xrif_with_deepseek_model(query)

                dump_json['Data Set'] = row['Data Set']
                dump_json['Experiment ID'] = row['Experiment ID']
                dump_json['Prompt ID'] = row['Prompt ID']
                dump_json['Prompt'] = query
                dump_json['Full LLM Response'] = str(llm_response)
                dump_json['XRIF Generated'] = xrif_response if isinstance(xrif_response, dict) else str(xrif_response)
                dump_json['Starting X'] = self.x_coord
                dump_json['Starting Y'] = self.y_coord
                dump_json['Expected Response'] = expected_response
                dump_json['Expected Response Type'] = row['Expected Response Type']
                expected_response = list(expected_response)
                if xrif_response and type(xrif_response) == dict:
                    if 'actions' in xrif_response.keys():
                        actions = xrif_response['actions']
                        if len(expected_response) == len(actions):
                            if row['Expected Response Type'] == 'Nav':
                                list_of_locations = [action['input']['name'] for action in actions if (action['action'] == 'navigate') and ('input' in action) and ('name' in action['input'])]
                                set_of_locations = set(list_of_locations)
                                set_of_expected_locations = set(expected_response)
                                if set_of_locations == set_of_expected_locations:
                                    note = "All locations are present in the response"
                                    dump_json['Notes'] = note
                            for i in range(len(xrif_response['actions'])):
                                if actions[i]['action'] == 'navigate' :
                                    if type(expected_response[i]) == 'str':
                                        if 'input' in actions[i]:
                                            if 'name' in actions[i]['input'].keys():
                                                if actions[i]['input']['name'] == expected_response[i] or (type(expected_response[i]) == 'list' and actions[i]['input']['name'] in expected_response[i]):
                                                    continue
                                                else:
                                                    error_message = f"Expected location name: {actions[i]['name']} does not match with the provided location name: {expected_response[i]}"
                                                    dump_json['Error Message'] = error_message
                                                    break
                                            else:
                                                error_message = f"Response Error Missing field: 'name' in action object {i}"
                                                dump_json['Error Message'] = error_message
                                                break
                                        else:
                                            error_message = f"Response Error Missing field: 'input' in action object {i}"
                                            dump_json['Error Message'] = error_message
                                            break
                                    else:
                                        error_message = f"Expected Response Error: Expected Response Type is not 'Nav' for action object {i}"
                                        dump_json['Error Message'] = error_message
                                        break
                                elif actions[i]['action'] == 'wait':
                                    if type(expected_response[i]) == 'tuple':
                                        if 'input' in actions[i]:
                                                if actions[i]['action'] != expected_response[i][0]:
                                                    error_message = f"Expected action at action object {i} : {expected_response[i][0]}  does not match with the provided action: {actions[i]['action']}"
                                                    dump_json['Error Message'] = error_message
                                                    break
                                                else:
                                                    if (int(actions[i]['input']))/60 == expected_response[i][1]:
                                                        error_message = f"Expected wait time: {actions[i]['input']} does not match with the provided wait time: {expected_response[i][1]}"
                                                        dump_json['Error Message'] = error_message
                                                        break
                                        else:
                                            error_message = f"Response Error Missing field: 'input' in action object {i}"
                                            dump_json['Error Message'] = error_message
                                            break
                                    else:
                                        error_message = f"Expected Response Error: Expected Response Type is not 'wait' for action object {i}"
                                        dump_json['Error Message'] = error_message
                                        break
                                elif actions[i]['action'] == 'speak':
                                    if type(expected_response[i]) == 'tuple':
                                        if ('action' in actions[i]) and ('input' in actions[i]):
                                            if actions[i]['action'] == expected_response[i][0] and (actions[i]['input'] == expected_response[i][1] or (type(expected_response[i]) == 'list' and actions[i]['input'] in expected_response[i])):
                                                continue
                                            else:
                                                error_message = f"Expected speak message: {expected_response[i]} does not match with the generated speak message: {actions[i]['input']} or Expected action: {expected_response[i][0]} does not match with the generated action: {actions[i]['action']}"
                                                dump_json['Error Message'] = error_message
                                                break
                                        else:
                                            error_message = f"Response Error Missing field: 'input' or 'action' in action object {i}"
                                            dump_json['Error Message'] = error_message
                                            break
                                    else:
                                        error_message = f"Expected Response Error: Expected Response Type is not correct for action object {i}"
                                        dump_json['Error Message'] = error_message
                                        break
                else:
                    error_message = "XRIF response is Invalid"
                    dump_json['Error Message'] = error_message
                
                if not os.path.exists(OUTPUT_DIR / output_folder):
                    os.makedirs(OUTPUT_DIR / output_folder)

                with open(OUTPUT_DIR / output_folder / f"Prompt_{row['Prompt ID']}_{now}.json", 'w') as file:
                    json.dump(dump_json, file, indent=4)
                pbar.update(1)

        return OUTPUT_DIR / output_folder

    def generate_xrif_with_google_model(self, query: str):
        xrif_response = None
        documents = self.retriever.invoke(query)
        num_waypoints_injected = len(documents) 
        doc_texts = "\n".join([doc.page_content for doc in documents])

        rendered_prompt = self.prompt_template.format(documents=doc_texts, query = query, starting_location = f"X-Coordinate: {self.x_coord}, Y-Coordinate: {self.y_coord}")

        llm_response = self.rag_chain.invoke({"documents": doc_texts, "query": query, "starting_location": f"X-Coordinate: {self.x_coord}, Y-Coordinate: {self.y_coord}"})

        try:
            xrif_response = llm_response.strip("```json\n```")
            xrif_response = json.loads(xrif_response)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON response: {e}")
            xrif_response = llm_response

        return xrif_response, llm_response, rendered_prompt, doc_texts, num_waypoints_injected

    def eval_xrif(self, *, eval_json: dict, xrif_gen: dict, xrif_ex: list, response_type):
        error_message = None
        dump_json = eval_json
        xrif_ex = list(xrif_ex)
        if xrif_gen and type(xrif_gen) == dict:
            if 'actions' in xrif_gen.keys():
                actions = xrif_gen['actions']
                if len(xrif_ex) == len(actions):
                    if response_type == 'Nav':
                        list_of_locations = [action['input']['name'] for action in actions if ('action' in action and action['action'] == 'navigate') and ('input' in action) and ('name' in action['input'])]
                        set_of_locations = set(list_of_locations)
                        set_of_expected_locations = set(xrif_ex)
                        if set_of_locations == set_of_expected_locations:
                            note = "All locations are present in the response"
                            dump_json['Notes'] = note
                    for i in range(len(xrif_gen['actions'])):
                        if 'action' in actions[i]:
                            if actions[i]['action'] == 'navigate' :
                                if type(xrif_ex[i]) is str:
                                    if 'input' in actions[i]:
                                        if 'name' in actions[i]['input'].keys():
                                            if actions[i]['input']['name'] == xrif_ex[i] or (type(xrif_ex[i]) == 'list' and actions[i]['input']['name'] in xrif_ex[i]):
                                                continue
                                            else:
                                                error_message = f"Expected location name: {actions[i]['input']['name']} does not match with the provided location name: {xrif_ex[i]}"
                                                dump_json['Error Message'] = error_message
                                                break
                                        else:
                                            error_message = f"Response Error Missing field: 'name' in action object {i}"
                                            dump_json['Error Message'] = error_message
                                            break
                                    else:
                                        error_message = f"Response Error Missing field: 'input' in action object {i}"
                                        dump_json['Error Message'] = error_message
                                        break
                                else:
                                    error_message = f"Expected Response Error: Expected Response Type is not 'Nav' for action object {i}"
                                    dump_json['Error Message'] = error_message
                                    break
                            elif actions[i]['action'] == 'wait':
                                if type(xrif_ex[i]) is tuple:
                                    if 'input' in actions[i]:
                                            if actions[i]['action'] != xrif_ex[i][0]:
                                                error_message = f"Expected action at action object {i} : {xrif_ex[i][0]}  does not match with the provided action: {actions[i]['action']}"
                                                dump_json['Error Message'] = error_message
                                                break
                                            else:
                                                if (int(actions[i]['input']))/60 == xrif_ex[i][1]:
                                                    error_message = f"Expected wait time: {actions[i]['input']} does not match with the provided wait time: {xrif_ex[i][1]}"
                                                    dump_json['Error Message'] = error_message
                                                    break
                                    else:
                                        error_message = f"Response Error Missing field: 'input' in action object {i}"
                                        dump_json['Error Message'] = error_message
                                        break
                                else:
                                    error_message = f"Expected Response Error: Expected Response Type is not 'wait' for action object {i}"
                                    dump_json['Error Message'] = error_message
                                    break
                            elif actions[i]['action'] == 'speak':
                                if type(xrif_ex[i]) is tuple:
                                    if ('action' in actions[i]) and ('input' in actions[i]):
                                        if actions[i]['action'] == xrif_ex[i][0] and (actions[i]['input'] == xrif_ex[i][1] or (type(xrif_ex[i]) == 'list' and actions[i]['input'] in xrif_ex[i])):
                                            continue
                                        else:
                                            error_message = f"Expected speak message: {xrif_ex[i]} does not match with the generated speak message: {actions[i]['input']} or Expected action: {xrif_ex[i][0]} does not match with the generated action: {actions[i]['action']}"
                                            dump_json['Error Message'] = error_message
                                            break
                                    else:
                                        error_message = f"Response Error Missing field: 'input' or 'action' in action object {i}"
                                        dump_json['Error Message'] = error_message
                                        break
                                else:
                                    error_message = f"Expected Response Error: Expected Response Type is not correct for action object {i}"
                                    dump_json['Error Message'] = error_message
                                    break
                        else:
                            error_message = f"Response Error Missing field: 'action' in action object {i}"
                            dump_json['Error Message'] = error_message
                            break
        else:
            error_message = "XRIF response is Invalid"
            dump_json['Error Message'] = error_message
        if not error_message:
            dump_json['Error Message'] = ""
        return dump_json
    
    def generate_xrif_with_openai_model(self, query: str):
        xrif_response = None
        documents = self.retriever.invoke(query)
        num_waypoints_injected = len(documents)
        doc_texts = "\n".join([doc.page_content for doc in documents])
        rendered_prompt = self.prompt_template.format(documents=doc_texts, query = query, starting_location = f"X-Coordinate: {self.x_coord}, Y-Coordinate: {self.y_coord}")

        llm_response = self.rag_chain.invoke({"documents": doc_texts, "query": query, "starting_location": f"X-Coordinate: {self.x_coord}, Y-Coordinate: {self.y_coord}"})

        try:
            xrif_response = llm_response.strip("```json\n```")
            xrif_response = json.loads(xrif_response)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON response: {e}")
            xrif_response = llm_response

        return xrif_response, llm_response, rendered_prompt, doc_texts, num_waypoints_injected

    def xrif_inference(self, query: str):
        if self.language_model_type == "Ollama":
            return self.generate_xrif_with_deepseek_model(query)
        elif self.language_model_type == "google":
            return self.generate_xrif_with_google_model(query)
        elif self.language_model_type == "OpenAI":
            return self.generate_xrif_with_openai_model(query)
        else:
            raise Exception("Invalid model type")
        
    def batch_experiment_run(self, df: pd.DataFrame, output_folder: Path, sleep_time: int = 0):
        num_rows = len(df)
        print(f"Processing {num_rows} test prompts...")
        with tqdm.tqdm(total=num_rows) as pbar:
            for index, row in df.iterrows():
                now = pd.Timestamp.now().strftime("%Y-%m-%d-%H-%M-%S")
                prompt_result = {}
                query = row['Prompt']
                self.x_coord = row['Starting X']
                self.y_coord = row['Starting Y']
                expected_response = row['Expected Response']
                xrif_response, llm_response, rendered_prompt, doc_texts, num_waypoints_injected = self.xrif_inference(query)
                prompt_result['rendered_prompt'] = rendered_prompt
                prompt_result['Waypoints Injected'] = doc_texts
                prompt_result['num_waypoints_injected'] = num_waypoints_injected
                prompt_result['Data Set'] = row['Data Set']
                prompt_result['Experiment ID'] = row['Experiment ID']
                prompt_result['Prompt ID'] = row['Prompt ID']
                prompt_result['Prompt'] = query
                prompt_result['Full LLM Response'] = str(llm_response)
                prompt_result['XRIF Generated'] = xrif_response if isinstance(xrif_response, dict) else str(xrif_response)
                prompt_result['Starting X'] = self.x_coord
                prompt_result['Starting Y'] = self.y_coord
                prompt_result['Expected Response'] = expected_response
                prompt_result['Expected Response Type'] = row['Expected Response Type']
                expected_response = list(expected_response)
                dump_json = self.eval_xrif(eval_json = prompt_result, xrif_gen = xrif_response, xrif_ex = expected_response, response_type = row['Expected Response Type'])
                if not os.path.exists(OUTPUT_DIR / output_folder):
                    os.makedirs(OUTPUT_DIR / output_folder)

                with open(OUTPUT_DIR / output_folder / f"Prompt_{row['Prompt ID']}_{now}.json", 'w') as file:
                    json.dump(dump_json, file, indent=4)

                if sleep_time > 0:
                    time.sleep(sleep_time)
                pbar.update(1)

        return OUTPUT_DIR / output_folder

In [10]:
def process_test_dataset(dataset: pd.DataFrame):
    dataset['Expected Response'] = dataset['Expected Response'].to_list()
    dataset.dropna(subset=['Prompt', 'Expected Response'], inplace=True)
    
    return dataset


def load_experiment_from_config(config_file: Path) -> Tuple[XRIFGenerator, pd.DataFrame, Path]:
    with open(config_file, 'r') as file:
        config_dict = safe_load(file)
        test_dataset_path = config_dict['test_dataset']

        test_dataset = pd.read_csv(test_dataset_path)
        output_folder = config_dict['output_folder']

        del config_dict['output_folder']
        del config_dict['test_dataset']

        test_dataset = process_test_dataset(test_dataset)
        
        return XRIFGenerator(**config_dict), test_dataset, output_folder

In [ ]:
google_xrif = XRIFGenerator(prompt_template="prompts/e7_xrif_actions_6.yaml", 
                            waypoints_csv="waypoint_datasets/uw_e7_floor1.csv",
                            language_model_type="google",
                            embed_model_type="HuggingFace",
                            language_model="gemini-2.0-flash",
                            embed_model="sentence-transformers/all-MiniLM-L6-v2",
                            chat_kwargs={"max_tokens": 500, "temperature": 0.2, "top_p": 0.2},
                            fetch_all=True)

Loading model: gemini-2.0-flash...
Processing 20 waypoints...


100%|██████████| 20/20 [00:00<00:00, 37516.14it/s]

Adding waypoints to vector store...
Creating retriever...
Creating prompt template...
Creating RAG chain...


In [105]:
openai_xrif = XRIFGenerator(prompt_template="prompts/e7_xrif_actions_6.yaml",
                            waypoints_csv="waypoint_datasets/uw_e7_floor1.csv",
                            language_model_type="OpenAI",
                            embed_model_type="HuggingFace",
                            language_model="gpt-4o-mini",
                            embed_model="sentence-transformers/all-MiniLM-L6-v2",
                            chat_kwargs={"num_predict": 500, "temperature": 0.2, "top_p": 0.2},
                            fetch_all=True)

Loading model: gpt-4o-mini...
Processing 20 waypoints...


100%|██████████| 20/20 [00:00<00:00, 38746.46it/s]

Adding waypoints to vector store...
Creating retriever...
Creating prompt template...
Creating RAG chain...


In [106]:
res = openai_xrif.generate_xrif_with_openai_model("Navigate to the nearest washroom")

In [107]:
res

({'actions': [{'action': 'speak', 'input': 'This waypoint does not exist'}]},
 '{\n  "actions": [\n    {\n      "action": "speak",\n      "input": "This waypoint does not exist"\n    }\n  ]\n}',
 '# Role\nYou control a robot that navigates through a building using a JSON instruction format. You have access to several pre-defined waypoints and functions. Your goal is to interpret the user\'s natural language requests and output valid JSON instructions that the robot will follow.\n\n# JSON Output Format\nEach output must be valid JSON (following RFC 8259) with the following structure:\n{\n  "actions": [\n    {\n      "action": "<action_type>",  // Must be one of: "navigate", "speak", "wait"\n      "input": <value>  // For "navigate": an object with "name", "x", and "y"; for "speak": a string; for "wait": a number (in seconds)\n    },\n    ...\n  ]\n}\nEnsure there are no trailing commas and all JSON rules are followed.\n\n# Context\n- Available Waypoints: \n  A waypoint called Shower B e

In [62]:
xrif_response, llm_response, render, docs, num_waypoints_injected = google_xrif.generate_xrif_with_google_model("Navigate to the nearest washroom")

In [ ]:
print(render)
print(docs)

In [49]:
llm_response

'```json\n{\n  "actions": [\n    {\n      "action": "speak",\n      "input": "This waypoint does not exist"\n    }\n  ]\n}\n```'

In [42]:
waypoints_count = 0
waypoints = pd.read_csv("waypoint_datasets/uw_e7_floor1.csv")
for index, row in waypoints.iterrows():
    keywords = row['Keywords']
    waypoints_count += 1

waypoints_count

20

In [85]:
experiment_generator, experiment_dataframe, output_folder = load_experiment_from_config(Path('experiment_configs/google/gemini_e7_hgf_embed.yaml'))

experiment_generator.batch_experiment_run(experiment_dataframe, output_folder, 7)

Loading model: gemini-2.0-flash...
Processing 20 waypoints...


100%|██████████| 20/20 [00:00<00:00, 38675.00it/s]


Adding waypoints to vector store...
Creating retriever...
Creating prompt template...
Creating RAG chain...
Processing 87 test prompts...


100%|██████████| 87/87 [12:39<00:00,  8.73s/it]


PosixPath('test_results/gemini_e7_hgf_embed')

In [ ]:
experiment_generator, experiment_dataframe, output_folder = load_experiment_from_config(Path('experiment_configs/deepseek/all_warehouse_deepseek14b.yaml'))

experiment_generator.batch_experiment_run(experiment_dataframe, output_folder, 0)

Loading model: deepseek-r1:14b...
Processing 24 waypoints...


100%|██████████| 24/24 [00:00<00:00, 11995.15it/s]


Adding waypoints to vector store...
Creating retriever...
Creating prompt template...
Creating RAG chain...
Processing 60 test prompts...


 68%|██████▊   | 41/60 [13:52<07:01, 22.19s/it]

Error parsing JSON response


 93%|█████████▎| 56/60 [19:07<01:23, 21.00s/it]

In [112]:
import os
from pathlib import Path
import json
import pandas as pd


for folder in os.listdir('test_results'):
    print(f"Results for {folder}...")
    if Path('test_results/' + folder).is_dir():
        json_list = []
        for file in os.listdir('test_results/' + folder):
            with open(Path('test_results/' + folder) / file, 'r') as f:
                json_list.append(json.load(f))
        
        df = pd.DataFrame(json_list)
        df.to_csv(f"test_results/{folder}.csv", index=False)

Results for eval_test...
Results for eval_test.csv...
Results for gemini_e7_hgf_embed.csv...
Results for all_e7_deepseek7b...
Results for gpt4o_mini_e7_hgf_embed.csv...
Results for all_e7_deepseek7b_hfembed.csv...
Results for all_e7_deepseek1.5b.csv...
Results for all_e7_deepseek14b.csv...
Results for all_e7_deepseek14b...
Results for gemini_e7_hgf_embed...
Results for all_e7_deepseek1.5b...
Results for all_e7_deepseek7b.csv...
Results for gpt4o_mini_e7_hgf_embed...
Results for all_e7_deepseek7b_hfembed...
